# Feature Validation Analysis - Scalp EEG

Here, we run through a validation analysis of scalp EEG data for the purposes of predicting epilepsy.

In [1]:
%load_ext lab_black

In [3]:
import numpy as np
import scipy
import scipy.io
import pandas as pd
from pathlib import Path
import os
import collections
from natsort import natsorted
import json
import pickle

from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.metrics import (
    brier_score_loss,
    roc_curve,
    average_precision_score,
    roc_auc_score,
    f1_score,
    balanced_accuracy_score,
    accuracy_score,
    auc,
    plot_precision_recall_curve,
    average_precision_score,
    precision_recall_curve,
    confusion_matrix,
)
from sklearn.inspection import permutation_importance
from sklearn.model_selection import (
    GroupKFold,
    GroupShuffleSplit,
    StratifiedShuffleSplit,
    ShuffleSplit,
    cross_validate,
)
from sklearn.utils import resample

from mne_bids import BIDSPath, get_entities_from_fname, get_entity_vals

# from rerf.rerfClassifier import rerfClassifier
from oblique_forests.sporf import ObliqueForestClassifier

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'oblique_forests'

In [4]:
from sklearn import preprocessing

enc = preprocessing.OrdinalEncoder()

# Setup Data Directory

In [6]:
jhroot = Path("/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/")
jeffroot = Path("/Users/adam2392/Johns Hopkins/Jefferson_Scalp - Documents/root/")

# not ready yet
upmcroot = Path("/Users/adam2392/Johns Hopkins/UPMC_Scalp - Documents/")

In [ ]:
root = Path("/Users/adam2392/OneDrive - Johns Hopkins/epilepsy_interictal")
# root = Path("/home/adam2392/hdd/Dropbox/epilepsy_interictal")
deriv_root = root / "derivatives"
source_root = root / "sourcedata"

session = "presurgery"
task = "interictal"
acquisition = "seeg"  # note not all subjects are actually sEEG
reference = "car"
deriv_chain = Path("sourcesink") / reference / "remove_artifact_wins"

deriv_path = deriv_root / deriv_chain

In [ ]:
def _load_subject_derivs(deriv_path, subjects=None, extension='.npy'):
    if subjects is None:
        subjects = get_entity_vals(deriv_path, 'subject')
    
    for subject in subjects:
        subj_path = deriv_path / f'sub-{subject}'